# M2モデルによる半教師あり学習
では，Model APIに用意されたクラスでは表現することの難しい，より複雑なモデルはどう実装すればよいでしょうか．

Pixyzでは，複雑なモデルに対応するためにLoss APIが用意されています．

出典: https://github.com/masa-su/pixyz/blob/master/examples/m2.ipynb

In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 128
epochs = 10
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [2]:
# https://github.com/wohlert/semi-supervised-pytorch/blob/master/examples/notebooks/datautils.py

from functools import reduce
from operator import __or__
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import MNIST
import numpy as np
from itertools import cycle

labels_per_class = 10
n_labels = 10

root = 'data'
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambd=lambda x: x.view(-1))])

mnist_train = MNIST(root=root, train=True, download=True, transform=transform)
mnist_valid = MNIST(root=root, train=False, transform=transform)

def get_sampler(labels, n=None):
    # Only choose digits in n_labels
    (indices,) = np.where(reduce(__or__, [labels == i for i in np.arange(n_labels)]))

    # Ensure uniform distribution of labels
    np.random.shuffle(indices)
    indices = np.hstack([list(filter(lambda idx: labels[idx] == i, indices))[:n] for i in range(n_labels)])

    indices = torch.from_numpy(indices)
    sampler = SubsetRandomSampler(indices)
    return sampler

# Dataloaders for MNIST
kwargs = {'num_workers': 1, 'pin_memory': True}
labelled = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                       sampler=get_sampler(mnist_train.train_labels.numpy(), labels_per_class),
                                       **kwargs)
unlabelled = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                         sampler=get_sampler(mnist_train.train_labels.numpy()), **kwargs)
validation = torch.utils.data.DataLoader(mnist_valid, batch_size=batch_size,
                                         sampler=get_sampler(mnist_valid.test_labels.numpy()), **kwargs)


### 0) Pixyzのimport
今回は，複雑なモデルの代表例としてm2モデルをLoss APIを使って実装します．

m2モデルのlossは
$$
-\sum _ { x , y \sim p _ { d a t a } ( x , y ) } \left[ E _ { q ( z | x , y ) } \left[ \log \frac { p ( x , z | y ) } { q ( z | x , y ) } \right] + \alpha \log q ( y | x ) \right] - \sum _ { x _ { u } \sim p _ { d a t a } \left( x _ { u } \right) } \left[ E _ { q ( z | x _ { u } , y ) q ( y | x _ { u } ) } \left[ \log \frac { p \left( x _ { u } , z | y \right) } { q ( z | x _ { u } , y ) q ( y | x _ { u } ) } \right] \right]
$$

で表され，第1項は，ラベル$y$が入手できる場合のloss，第2項はラベルが入手できない場合のlossになっています．

事前分布とエンコーダには正規分布，デコーダにはベルヌーイ分布を利用します．
- 事前分布は平均が0，標準偏差が1の標準正規分布を用います(なのでlocとscaleが定数)

さらに，m2モデルでは，カテゴリ分布を用いて，識別器 $p(y|x)$ も訓練します．
- 今回は，`RelaxedCategorical`を用いることにします．

VAEのときと同様に，分布の中身を書いていきましょう．各分布のforwardの返り値が，分布のパラメータをdictにしたものになるようにします．

In [3]:
from pixyz.distributions import Normal, Bernoulli, RelaxedCategorical
from pixyz.models import Model
from pixyz.losses import ELBO, NLL

In [4]:
x_dim = 784
y_dim = 10
z_dim = 64


# inference model q(z|x,y)
class Inference(Normal):
    def __init__(self):
        super().__init__(cond_var=["x","y"], var=["z"], name="q")

        self.fc1 = nn.Linear(x_dim+y_dim, 512)
        self.fc21 = nn.Linear(512, z_dim)
        self.fc22 = nn.Linear(512, z_dim)

    def forward(self, x, y):
        h = F.relu(self.fc1(torch.cat([x, y], 1)))
        return {"loc": self.fc21(h), "scale": F.softplus(self.fc22(h))}

    
# generative model p(x|z,y)    
class Generator(Bernoulli):
    def __init__(self):
        super().__init__(cond_var=["z","y"], var=["x"], name="p")

        self.fc1 = nn.Linear(z_dim+y_dim, 512)
        self.fc2 = nn.Linear(512, x_dim)

    def forward(self, z, y):
        h = F.relu(self.fc1(torch.cat([z, y], 1)))
        return {"probs": torch.sigmoid(self.fc2(h))}

# classifier p(y|x)
class Classifier(RelaxedCategorical):
    def __init__(self):
        super(Classifier, self).__init__(cond_var=["x"], var=["y"], temperature=0.1, name="p")
        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, y_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.softmax(self.fc2(h), dim=1)
        return {"probs": h}
    
# prior model p(z)
loc = torch.tensor(0.).to(device)
scale = torch.tensor(1.).to(device)
prior = Normal(loc=loc, scale=scale, var=["z"], dim=z_dim, name="prior")    

分布のインスタンスを作って，指定のdeviceに載せます

In [5]:
# distributions for supervised learning
p = Generator().to(device)
q = Inference().to(device)
f = Classifier().to(device)
p_joint = p * prior

Pixyzでは，分布をprintすることで，その分布の中身のネットワークがどうなっているかを確認できるのでしたね．

実際にあっているか確認してみましょう．

In [6]:
print(p_joint)
print(q)
print(f)

Distribution:
  p(x,z|y) = p(x|z,y)prior(z)
Network architecture:
  prior(z) (Normal): Normal()
  p(x|z,y) (Bernoulli): Generator(
    (fc1): Linear(in_features=74, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=784, bias=True)
  )
Distribution:
  q(z|x,y) (Normal)
Network architecture:
  Inference(
    (fc1): Linear(in_features=794, out_features=512, bias=True)
    (fc21): Linear(in_features=512, out_features=64, bias=True)
    (fc22): Linear(in_features=512, out_features=64, bias=True)
  )
Distribution:
  p(y|x) (RelaxedCategorical)
Network architecture:
  Classifier(
    (fc1): Linear(in_features=784, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=10, bias=True)
  )


意図した通りになっていそうですね．

次に，ラベルがない場合のときのLossを書いていきましょう．

Pixyzでは，同じネットワークのパラメータを共有する分布を `replace_var`メソッドを使って作ることができます．
- この時は，以下のように，元の確率変数を新たな確率変数で置き換えるように書きます．

In [7]:
# distributions for unsupervised learning
q_u = q.replace_var(x="x_u", y="y_u")
p_u = p.replace_var(x="x_u", y="y_u")
f_u = f.replace_var(x="x_u", y="y_u")

ラベルのない場合には，$x_u$が与えられたときの$y$と$z$の同時分布$q(z,y_u|x_u)$と，$y_u$が与えられたときの$x_u$と$z$の同時分布$p(x_u,z|y_u)$が必要ですので，これらの同時分布を作りましょう．

Pixyzでは同時分布は分布同士の掛け算で表現できるのでした．

In [8]:
q_u = q_u * f_u
p_joint_u = p_u * prior

確認してみましょう

In [9]:
print(p_joint_u)
print(q_u)
print(f_u)

Distribution:
  p(x_u,z|y_u) = p(x_u|z,y_u)prior(z)
Network architecture:
  prior(z) (Normal): Normal()
  p(x_u|z,y_u) (Bernoulli): Generator(
    (fc1): Linear(in_features=74, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=784, bias=True)
  )
Distribution:
  p(z,y_u|x_u) = q(z|x_u,y_u)p(y_u|x_u)
Network architecture:
  p(y_u|x_u) (RelaxedCategorical): Classifier(
    (fc1): Linear(in_features=784, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=10, bias=True)
  )
  q(z|x_u,y_u) (Normal): Inference(
    (fc1): Linear(in_features=794, out_features=512, bias=True)
    (fc21): Linear(in_features=512, out_features=64, bias=True)
    (fc22): Linear(in_features=512, out_features=64, bias=True)
  )
Distribution:
  p(y_u|x_u) (RelaxedCategorical)
Network architecture:
  Classifier(
    (fc1): Linear(in_features=784, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=10, bias=True)
  )


意図した通り， 
$$p(x_u,z|y_u) = p(x_u|z,y_u)prior(z)$$
$$ p(z,y_u|x_u) = q(z|x_u,y_u)p(y_u|x_u)$$
となっており，2つの分布の積として因数分解される同時分布になっていることが確認できました．

### 2) 目的関数の設定
m2モデルのlossは
$$
-\sum _ { x , y \sim p _ { d a t a } ( x , y ) } \left[ E _ { q ( z | x , y ) } \left[ \log \frac { p ( x , z | y ) } { q ( z | x , y ) } \right] + \alpha \log q ( y | x ) \right] - \sum _ { x _ { u } \sim p _ { d a t a } \left( x _ { u } \right) } \left[ E _ { q ( z | x _ { u } , y ) q ( y | x _ { u } ) } \left[ \log \frac { p \left( x _ { u } , z | y \right) } { q ( z | x _ { u } , y ) q ( y | x _ { u } ) } \right] \right]
$$
で表されるのでした．

それではこのlossを表現していきましょう．

lossの式を見ると，$
\sum _ { x , y \sim p _ { d a t a } ( x , y ) } \left[ E _ { q ( z | x , y ) } \left[ \log \frac { p ( x , z | y ) } { q ( z | x , y ) } \right]  \right]$と，
$\sum _ { x _ { u } \sim p _ { d a t a } \left( x _ { u } \right) } \left[ E _ { q ( z | x _ { u } , y ) q ( y | x _ { u } ) } \left[ \log \frac { p \left( x _ { u } , z | y \right) } { q ( z | x _ { u } , y ) q ( y | x _ { u } ) } \right] \right]$の部分は，それぞれラベルがあるとき，ないときのELBOになっていることがわかります．

そのため，この2つには，`ELBO` Lossを使ってあげればよいでしょう．
- ELBO Lossの詳細は， https://docs.pixyz.io/en/latest/losses.html#lower-bound にあります．

In [10]:
elbo_u = ELBO(p_joint_u, q_u)
elbo = ELBO(p_joint, q)

残りの項で先頭のマイナスを，シグマの内側に入れたもの$\sum _ { x , y \sim p _ { d a t a } ( x , y ) } \left[ -\alpha \log q ( y | x ) \right]$は，$q ( y | x ) \$の負の対数尤度(NLL)となっています．
そのためこの項には`NLL` Lossを使ってあげればよいでしょう．
- NLL Lossの詳細は， https://docs.pixyz.io/en/latest/losses.html#nll にあります．

In [11]:
nll = NLL(f)

全体のLossは以上の3つのLoss(`elbo_u`，`elbo`,`nll`)を組み合わせたものになっています．

これらの四則演算によって，最終的なLossを作り出しましょう．

【ポイント！】 PixyzではLoss同士の四則演算ができます．

In [12]:
rate = 1 * (len(unlabelled) + len(labelled)) / len(labelled)

loss_cls = -elbo_u.mean() -elbo.mean() + (rate * nll).mean() 

それでは，このモデルを学習する前に，Lossが本当に欲しいものかどうかチェックしてみましょう．

【ポイント！】 Lossをprintするとそのロスの式を確認できます．

In [13]:
print(loss_cls)

-(mean(E_p(z,y_u|x_u)[log p(x_u,z|y_u)/p(z,y_u|x_u)])) - mean(E_q(z|x,y)[log p(x,z|y)/q(z|x,y)]) + mean(-log p(y|x) * 470.0)


意図したものになっていますね．

Lossクラスを使って，自分でLossを定義した時は，ModelクラスのModelに必要なものを渡します．
- 以下のように，学習時のloss，テスト時のloss，モデルを構成する分布を渡します.
- モデルクラスの詳細は， https://docs.pixyz.io/en/latest/models.html#model にあります，

In [14]:
model = Model(loss_cls,test_loss=nll.mean(),
              distributions=[p, q, f], optimizer=optim.Adam, optimizer_params={"lr":1e-3})

構築したモデルの中身をチェックしてみます．

In [15]:
print(model)

Distributions (for training): 
  p(x|z,y), q(z|x,y), p(y|x) 
Loss function: 
  -(mean(E_p(z,y_u|x_u)[log p(x_u,z|y_u)/p(z,y_u|x_u)])) - mean(E_q(z|x,y)[log p(x,z|y)/q(z|x,y)]) + mean(-log p(y|x) * 470.0) 
Optimizer: 
  Adam (
  Parameter Group 0
      amsgrad: False
      betas: (0.9, 0.999)
      eps: 1e-08
      lr: 0.001
      weight_decay: 0
  )


大丈夫そうですね．

では，学習を始めましょう．

### 学習してみる
それでは実際に学習してみましょう．

モデルクラスには，`train()`メソッドが用意されており，その引数に入力を与えるだけで，内部でlossを`backward()`して,パラメータを更新(`optimizer.step()`)してくれます．

`test()`メソッドでは，引数として与えられた入力に対するLossを返します．

In [16]:
def train(epoch):
    train_loss = 0
    for (x, y), (x_u, y_u) in tqdm(zip(cycle(labelled), unlabelled), total=len(unlabelled)):
        x = x.to(device)
        y = torch.eye(10)[y].to(device)
        x_u = x_u.to(device)        
        loss = model.train({"x": x, "y": y, "x_u": x_u})
        train_loss += loss
        
    train_loss = train_loss * unlabelled.batch_size / len(unlabelled.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    
    return train_loss

In [17]:
def test(epoch):
    test_loss = 0
    correct = 0
    total = 0    
    for x, y in validation:
        x = x.to(device)
        y = torch.eye(10)[y].to(device)        
        loss = model.test({"x": x, "y": y})
        test_loss += loss
        
        pred_y = f.sample_mean({"x": x})
        total += y.size(0)
        correct += (pred_y.argmax(dim=1) == y.argmax(dim=1)).sum().item()      

    test_loss = test_loss * validation.batch_size / len(validation.dataset)
    test_accuracy = 100 * correct / total
    print('Test loss: {:.4f}, Test accuracy: {:.4f}'.format(test_loss, test_accuracy))
    return test_loss, test_accuracy

In [18]:
writer = SummaryWriter()

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss, test_accuracy = test(epoch)

    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)
    writer.add_scalar('test_accuracy', test_accuracy, epoch)    
    
writer.close()

100%|██████████| 469/469 [00:47<00:00, 10.49it/s]

Epoch: 1 Train loss: 328.9215


Test loss: 1.0270, Test accuracy: 76.5000


100%|██████████| 469/469 [00:47<00:00, 11.10it/s]


Epoch: 2 Train loss: 213.4401
Test loss: 1.4781, Test accuracy: 74.7000


...

100%|██████████| 469/469 [00:49<00:00,  9.41it/s]

Epoch: 9 Train loss: 179.3192


Test loss: 2.1873, Test accuracy: 77.9000


100%|██████████| 469/469 [00:48<00:00,  9.75it/s]


Epoch: 10 Train loss: 178.3697
Test loss: 2.7381, Test accuracy: 75.2500
